In [222]:
import os #Files in listen abgreifen
import pandas as pd
import camelot # um Tabellen im PDFs zu erkennen
import PyPDF2 # um Text aus PDFs zu lesen
import re

In [223]:
regex_gnummer = r"(\d\d\.\S+)"
regex_datum = r"\d\d\.\d\d\.\d\d\d\d \d\d\:\d\d\:\d\d"

In [224]:
files = os.listdir("Sessionsprotokolle")

In [225]:
files

['SR_5117_2212.pdf',
 'SR_5105_2009.pdf',
 'SR_5110_2106.pdf',
 'SR_5107_2012.pdf',
 'SR_5119_2304.pdf',
 'SR_5112_2112.pdf',
 'SR_5118_2303.pdf',
 'SR_5115_2206.pdf',
 'SR_5102_2003.pdf',
 'SR_5111_2109.pdf',
 'SR_5116_2209.pdf',
 'SR_5113_2203.pdf',
 'SR_5104_2006.pdf',
 'SR_5121_2306.pdf',
 'SR_5108_2103.pdf']

In [226]:
master_df_list = []

for file in files:

    print(file)
    
    session = file
    
    dokument = PyPDF2.PdfReader("Sessionsprotokolle/" + str(file))
    
    seitenliste = []

    laenge = len(dokument.pages)

    for seite in range(laenge):
        inhalt = dokument.pages[seite].extract_text()
        if inhalt[:8] == "Geschäft":
            nummer = seite
            seitenliste.append(nummer)

    for seite in seitenliste:
        inhalt = dokument.pages[seite].extract_text()
        datenliste = inhalt.split("\n")
    
        for zeile in range(len(datenliste)):
            if "Abstimmung vom" in datenliste[zeile]:
        
                datum = re.findall(regex_datum, datenliste[zeile])[0]
            
                print(datum)
    
        for zeile in range(len(datenliste)):
            if "Geschäft / Objet:" in datenliste[zeile]:
                geschäftszeile = datenliste[(zeile+1)]
            
                gnummer = (re.findall(regex_gnummer, geschäftszeile))[0]
                geschäft = geschäftszeile.strip(str(gnummer)+" ")
                
                print(gnummer)
                print(geschäft)
                print(" ")
                
        for zeile in range(len(datenliste)):
            if "Gegenstand" in datenliste[zeile]:
                gegenstand = str(datenliste[zeile]).replace("Gegenstand / Objet du vote: ","")
            
                
                
        for zeile in range(len(datenliste)):
            if "+ Ja / oui / si " in datenliste[zeile]:
                jastimmen = int(str(datenliste[zeile]).replace("+ Ja / oui / si ",""))

        for zeile in range(len(datenliste)):    
            if "- Nein / non / no " in datenliste[zeile]:
                neinstimmen = int(str(datenliste[zeile]).replace("- Nein / non / no ",""))

        if jastimmen > neinstimmen: 
            resultat ="+"

        if jastimmen < neinstimmen: 
            resultat ="-"
            
        if abs(jastimmen - neinstimmen) <= 2:
            knapp = "Ja"
            
        if abs(jastimmen - neinstimmen) > 2:
            knapp = "Nein"
      
        # So holen wir die richtigen 24 Elemente fürs Abstimmungsverhalten aus der Tabelle
        try:
            tabellenanfang = datenliste.index(' ')+1
        
        #das ist der Ausnahmefall: So suchen wir die Abstimmungszeile und nehmen die nächste
        except:
            def first_substring(strings, substring):
                return next(i for i, string in enumerate(strings) if substring in string)

            tabellenanfang = first_substring(datenliste, 'Abstimmung vom / Vote du:')
        
        tabelle = datenliste[tabellenanfang:(tabellenanfang+24)]
        
        # So flicken wir die Gmür-Schnönenberger:
        tabelle.remove("Gmür-")
        tabelle = [w.replace('SchönenbergerAndrea', 'Gmür-Schönenberger Andrea') for w in tabelle]
        
        # und jetzt trennen wir jede Zeile beim ersten Auftauchen eines Kantons (jenes mit Leerschlag)
        tabelle_getrennt = []
        kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

        for paar in tabelle:
            for kanton in kantonsliste:
                trenner = kanton + " "
                if trenner in paar:
                    trennung = paar.split(trenner)
            
                    # (dann müssen wir den bei wplit verloren gegangenen Kanton, den Trenner, wieder hinzufügen)
                    trennung[0] = trennung[0]+kanton
            
                    tabelle_getrennt.append(trennung[0])
                    tabelle_getrennt.append(trennung[1])
                    
        for eintrag in tabelle_getrennt:
            eintrag_neu = eintrag.rsplit(" ", 2)
            
            name = eintrag_neu[0]
            kanton = eintrag_neu[2]
            verhalten = eintrag_neu[1]
            
            mini_dict = {
            "Session" : session,
            "Datum" : datum,
            "Nummer" : gnummer,
            "Geschäft" : geschäft,
            "Gegenstand" : gegenstand,
            "Resultat" : resultat,
            "knapp?" : knapp,
            "Politiker" : name,
            "Kanton" : kanton,
            "Verhalten" : verhalten}
    
            master_df_list.append(mini_dict)

SR_5117_2212.pdf
01.12.2022 10:36:05
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 10:59:30
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:13:52
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:14:43
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:33:33
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:44:56
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:45:35
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. Einführung des Monismus
 
01.12.2022 11:46:16
09.528-1
Pa. Iv. Humbel. Finanzierung der Gesundheitsleistungen aus einer Hand. E

05.12.2022 17:15:17
22.041-0
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 17:30:27
22.041-1
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 17:31:42
22.041-1
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 17:33:30
22.041-1
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 17:43:44
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 18:01:20
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 18:07:34
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 18:10:12
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 18:12:16
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 
05.12.2022 18:30:33
22.041-2
Voranschlag 2023 mit integriertem Aufgaben- und Finanzplan 2024-2026
 


13.12.2022 11:02:57
22.4252
Po. WAK-SR. Wettbewerbssituation im Lebensmittelmarkt
 
SR_5105_2009.pdf
25.09.2020 08:35:36
17.059-3
Datenschutzgesetz. Totalrevision und Änderung weiterer Erlasse zum Datenschutz
 
07.09.2020 19:05:09
17.071-1
Totalrevision des CO2-Gesetzes nach 202
 
07.09.2020 19:14:17
17.071-1
Totalrevision des CO2-Gesetzes nach 202
 
25.09.2020 08:36:06
17.071-1
Totalrevision des CO2-Gesetzes nach 202
 
22.09.2020 10:43:54
18.061-1
Doppelbesteuerung. Abkommen zwischen der Schweiz und Saudi-Arabien
 
25.09.2020 08:36:39
18.061-1
Doppelbesteuerung. Abkommen zwischen der Schweiz und Saudi-Arabien
 
25.09.2020 08:37:18
18.071-1
Terrorismus und organisierte Kriminalität. Übereinkommen des Europarates
 
25.09.2020 08:38:01
18.096-1
Für sauberes Trinkwasser und gesunde Nahrung - Keine Subventionen für den Pestizid- und de
 
09.09.2020 11:43:05
19.021-1
Betäubungsmittelgesetz. Änderung
 
25.09.2020 08:38:40
19.021-1
Betäubungsmittelgesetz. Änderung
 
25.09.2020 08:39:09
19.025

14.09.2020 20:19:56
20.057-1
AVIG. Zusatzfinanzierung der Arbeitslosenversicherung
 
14.09.2020 20:20:38
20.057-1
AVIG. Zusatzfinanzierung der Arbeitslosenversicherung
 
22.09.2020 08:20:09
20.057
AVIG. Zusatzfinanzierung der Arbeitslosenversicherung
 
25.09.2020 08:49:01
20.057-1
AVIG. Zusatzfinanzierung der Arbeitslosenversicherung
 
10.09.2020 16:36:45
20.058-1
Bundesgesetz über die gesetzlichen Grundlagen für Verordnungen des Bundesrates zur Bewältigung der
 
10.09.2020 17:18:18
20.058-1
Bundesgesetz über die gesetzlichen Grundlagen für Verordnungen des Bundesrates zur Bewältigung der
 
10.09.2020 18:13:22
20.058-1
Bundesgesetz über die gesetzlichen Grundlagen für Verordnungen des Bundesrates zur Bewältigung der
 
10.09.2020 18:32:43
20.058-1
Bundesgesetz über die gesetzlichen Grundlagen für Verordnungen des Bundesrates zur Bewältigung der
 
10.09.2020 18:33:33
20.058-1
Bundesgesetz über die gesetzlichen Grundlagen für Verordnungen des Bundesrates zur Bewältigung der
 
10.09.2020 1

03.12.2020 09:35:38
20.037-1
FIPOI. Finanzhilfen an die ITU in Genf
 
03.12.2020 09:36:22
20.037-1
FIPOI. Finanzhilfen an die ITU in Genf
 
08.12.2020 10:40:31
20.038-2
Massnahmenpaket zugunsten der Medien
 
18.12.2020 08:37:11
20.038-2
Massnahmenpaket zugunsten der Medien
 
07.12.2020 18:34:52
20.041-1
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:36:07
20.041-1
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:37:13
20.041-1
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:38:01
20.041-1
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:40:20
20.041-2
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:55:12
20.041-4
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
07.12.2020 18:56:24
20.041-5
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
16.12.2020 09:11:30
20.041-1
Voranschlag 2021 mit integriertem Aufgaben- und Finanzplan
 
16.1

...

02.03.2022 10:21:10
21.044-2
Keine Massentierhaltung in der Schweiz (Massentierhaltungsinitiative). Volksinitiative und direkter
 
02.03.2022 10:22:59
21.044-1
Keine Massentierhaltung in der Schweiz (Massentierhaltungsinitiative). Volksinitiative und direkter
 
18.03.2022 08:28:36
21.044-1
Keine Massentierhaltung in der Schweiz (Massentierhaltungsinitiative). Volksinitiative und direkter
 
28.02.2022 16:43:43
21.046-1
Veloweggesetz
 
18.03.2022 08:29:08
21.046-1
Veloweggesetz
 
18.03.2022 08:29:42
21.049-1
Gentechnikgesetz. Änderung
 
18.03.2022 08:30:18
21.053-1
Mobilität von Dienstleistungserbringern. Abkommen zwischen der Schweiz und dem Vereinigten Königreich
 
03.03.2022 10:45:18
21.054-1
PUBLICA-Gesetz. Änderung
 
18.03.2022 08:30:50
21.054-1
PUBLICA-Gesetz. Änderung
 
18.03.2022 08:31:36
21.056-1
Abkommen über die Besteuerung der Grenzgängerinnen und Grenzgänger und Protokoll zur Änderung des
 
18.03.2022 08:32:10
21.058-1
Soziale Sicherheit. Abkommen mit Tunesien
 
03.03.2022 1

04.06.2020 11:05:51
19.060-1
Doppelbesteuerung. Abkommen mit Schweden
 
19.06.2020 08:38:53
19.060-1
Doppelbesteuerung. Abkommen mit Schweden
 
04.06.2020 11:06:38
19.061-1
Doppelbesteuerung. Abkommen mit Norwegen
 
19.06.2020 08:39:33
19.061-1
Doppelbesteuerung. Abkommen mit Norwegen
 
04.06.2020 11:07:23
19.062-1
Doppelbesteuerung. Abkommen mit Neuseeland
 
19.06.2020 08:40:16
19.062-1
Doppelbesteuerung. Abkommen mit Neuseeland
 
03.06.2020 10:54:14
19.064-1
Zahlungsrahmen für die Förderung des alpenquerenden Schienengüterverkehrs. Änderung
 
03.06.2020 10:55:14
19.064-1
Zahlungsrahmen für die Förderung des alpenquerenden Schienengüterverkehrs. Änderung
 
18.06.2020 09:26:16
19.067-1
Vorläuferstoffgesetz
 
02.06.2020 16:51:22
19.068
Schutz des Menschen bei der automatischen Verarbeitung personenbezogener Daten. Übereinkommen
 
19.06.2020 08:41:06
19.068-1
Schutz des Menschen bei der automatischen Verarbeitung personenbezogener Daten. Übereinkommen
 
16.06.2020 10:52:28
19.070-1
EHB-G

18.06.2020 12:23:31
20.038-1
Massnahmenpaket zugunsten der Medien
 
18.06.2020 13:01:19
20.038-1
Massnahmenpaket zugunsten der Medien
 
18.06.2020 16:29:54
20.038-1
Massnahmenpaket zugunsten der Medien
 
18.06.2020 16:32:21
20.038-1
Massnahmenpaket zugunsten der Medien
 
18.06.2020 16:35:10
20.038-1
Massnahmenpaket zugunsten der Medien
 
03.06.2020 08:57:34
20.040-1
Dringliche Änderung des Epidemiengesetzes angesichts der COVID-19-Krise (Proximity-Tracing-System)
 
10.06.2020 09:07:44
20.040-1
Dringliche Änderung des Epidemiengesetzes angesichts der COVID-19-Krise (Proximity-Tracing-System)
 
19.06.2020 08:45:39
20.040-1
Dringliche Änderung des Epidemiengesetzes angesichts der COVID-19-Krise (Proximity-Tracing-System)
 
04.06.2020 10:09:15
20.042
Voranschlag 2020. Nachtrag IIa
 
04.06.2020 10:10:10
20.042
Voranschlag 2020. Nachtrag IIa
 
04.06.2020 10:10:58
20.042
Voranschlag 2020. Nachtrag IIa
 
04.06.2020 10:14:07
20.042
Voranschlag 2020. Nachtrag IIa
 
SR_5121_2306.pdf
05.06.2023 16

12.06.2023 17:58:46
22.491
Pa. Iv. Engler. Frühinvalide Bezügerinnen und Bezüger einer ausserordentlichen IV-Rente sollen den
 
14.06.2023 12:33:08
22.3883
Mo. WAK-NR. Schaffung einer Taskforce für die Sperrung von russischen und belorussischen
 
15.06.2023 11:27:37
22.3884
Mo. WAK-NR. Einführung eines Online-Preisrechners für Treibstoffe
 
31.05.2023 08:51:44
23.003-1
Staatsrechnung
 
31.05.2023 08:52:42
23.003-2
Staatsrechnung
 
31.05.2023 08:53:28
23.003-3
Staatsrechnung
 
31.05.2023 09:29:41
23.007
Voranschlag 2023. Nachtrag I
 
31.05.2023 09:48:27
23.007
Voranschlag 2023. Nachtrag I
 
31.05.2023 09:50:16
23.007-3
Voranschlag 2023. Nachtrag I
 
31.05.2023 09:50:58
23.007-3
Voranschlag 2023. Nachtrag I
 
31.05.2023 09:51:59
23.007-4
Voranschlag 2023. Nachtrag I
 
06.06.2023 10:13:30
23.007-2
Voranschlag 2023. Nachtrag I
 
13.06.2023 09:11:13
23.007
Voranschlag 2023. Nachtrag I
 
15.06.2023 08:44:12
23.007-3
Voranschlag 2023. Nachtrag I
 
08.06.2023 11:33:17
23.023-1
Transplantations

19.03.2021 08:29:58
20.048-1
Lissabonner Abkommen über Ursprungsbezeichnungen und geografische Angaben. Genehmigung
 
08.03.2021 16:41:53
20.060-1
Betäubungsmittelgesetz. Änderung (Cannabisarzneimittel)
 
19.03.2021 08:30:32
20.060-1
Betäubungsmittelgesetz. Änderung (Cannabisarzneimittel)
 
02.03.2021 08:27:33
20.065-1
Doppelbesteuerung. Abkommen mit Kuwait
 
19.03.2021 08:31:07
20.065-1
Doppelbesteuerung. Abkommen mit Kuwait
 
02.03.2021 08:28:27
20.066-1
Doppelbesteuerung. Abkommen mit Bahrain
 
19.03.2021 08:31:43
20.066-1
Doppelbesteuerung. Abkommen mit Bahrain
 
02.03.2021 08:47:28
20.067-1
Administrative Erleichterungen und Entlastung des Bundeshaushalts. Bundesgesetz
 
19.03.2021 08:32:15
20.067-1
Administrative Erleichterungen und Entlastung des Bundeshaushalts. Bundesgesetz
 
01.03.2021 17:07:58
20.070-1
Weiterentwicklungen des Schengen-Besitzstands. Genehmigung und Umsetzung der Notenaustausche
 
19.03.2021 08:32:59
20.070-1
Weiterentwicklungen des Schengen-Besitzstands. Gene

In [227]:
master_df = pd.DataFrame(master_df_list)
master_df

,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,Politiker,Kanton,Verhalten
0,SR_5117_2212.pdf,01.12.2022 10:36:05,09.528-1,Pa. Iv. Humbel. Finanzierung der Gesundheitsle...,"Art. 21 Abs. 1, Abs. 2 Bst. a - d, Abs. 3, 4 u...",+,Bauer Philippe,NE,+
1,SR_5117_2212.pdf,01.12.2022 10:36:05,09.528-1,Pa. Iv. Humbel. Finanzierung der Gesundheitsle...,"Art. 21 Abs. 1, Abs. 2 Bst. a - d, Abs. 3, 4 u...",+,Knecht Hansjörg,AG,+
2,SR_5117_2212.pdf,01.12.2022 10:36:05,09.528-1,Pa. Iv. Humbel. Finanzierung der Gesundheitsle...,"Art. 21 Abs. 1, Abs. 2 Bst. a - d, Abs. 3, 4 u...",+,Baume-Schneider Elisabeth,JU,-
3,SR_5117_2212.pdf,01.12.2022 10:36:05,09.528-1,Pa. Iv. Humbel. Finanzierung der Gesundheitsle...,"Art. 21 Abs. 1, Abs. 2 Bst. a - d, Abs. 3, 4 u...",+,Kuprecht Alex,SZ,+
4,SR_5117_2212.pdf,01.12.2022 10:36:05,09.528-1,Pa. Iv. Humbel. Finanzierung der Gesundheitsle...,"Art. 21 Abs. 1, Abs. 2 Bst. a - d, Abs. 3, 4 u...",+,Bischof Pirmin,SO,-
...,...,...,...,...,...,...,...,...,...
65951,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Zanetti Roberto,SO,+
65952,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Juillard Charles,JU,+
65953,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Z'graggen Heidi,UR,+
65954,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Knecht Hansjörg,AG,+


In [253]:
master_df[master_df["Session"] == "SR_5117_2212.pdf"]["Verhalten"].value_counts()

+    5361
-    1233
0     655
=     184
P     133
E     109
        7
Name: Verhalten, dtype: int64

In [228]:
master_df["Datum"].value_counts().to_frame().tail(5)

,Datum
06.12.2021 18:20:16,46
26.09.2022 17:13:34,44
13.06.2022 16:38:17,44
13.06.2023 08:39:21,44
22.09.2022 15:04:05,44


## Loop, der die Standesstimme auswertet

In [229]:
# Das hier generiert uns eine Liste mit allen Geschäften und allen Kantonen:
geschaeftsliste = master_df["Datum"].value_counts().index.to_list()
kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

In [230]:
# So schalte ich die Fehlermeldung aus
pd.options.mode.chained_assignment = None  # default='warn'

In [269]:
df_komplett = pd.DataFrame()

for geschaeft in geschaeftsliste:
    geschaeft_df = master_df[master_df["Datum"] == geschaeft]
    
    for kanton in kantonsliste:
        kanton_df = geschaeft_df[geschaeft_df["Kanton"] == kanton]
        standesstimme = kanton_df["Verhalten"].value_counts().to_dict()
        position_gleiche = kanton_df.index.to_list()
        
        for zeile in position_gleiche:
    
            #hier erledigen wir jene Kantone, die nur einen EIntrag haben
            if len(kanton_df) <= 1:
                kanton_df.loc[zeile, "Standesstimme"] = "Halbkanton"
     
            #hier erledigen wir jene, die aus verschiedenen Gründen nicht vollzählig waren
            elif "E" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "P" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "0" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
   
            #hier kommen jene, die gelich abgstimmt haben
            elif "+" in standesstimme:
                if standesstimme["+"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dafür"
                if standesstimme["+"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "-" in standesstimme:
                if standesstimme["-"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dagegen"
                if standesstimme["-"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "=" in standesstimme:
                if standesstimme["="] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide enthalten"
                if standesstimme["="] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
        
            #und schliesslich die anderen
            else:
                kanton_df.loc[zeile, "Standesstimme"] = "leer"
                
            if len(position_gleiche) > 1:
                if kanton_df.loc[position_gleiche[0], "Verhalten"] == "0" and kanton_df.loc[position_gleiche[1], "Verhalten"] == "0":
                    kanton_df.loc[position_gleiche[0], "Standesstimme"] = "beide absent"
                    kanton_df.loc[position_gleiche[1], "Standesstimme"] = "beide absent"
                
            for index, row in kanton_df.iterrows():
                if row["Verhalten"] == row["Resultat"]:
                    kanton_df.loc[index,"siegreich"] = "Ja"
                elif row["Verhalten"] != row["Resultat"]:
                    kanton_df.loc[index,"siegreich"] = "Nein"
                elif row["Verhalten"] == "0" or row["Verhalten"] == "P" or row["Verhalten"] == "E":
                    kanton_df.loc[index,"siegreich"] = "---"
                
        df_komplett = df_komplett.append(kanton_df)

In [271]:
df_komplett.tail(50)

,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,Politiker,Kanton,Verhalten,Standesstimme,siegreich
62756,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Ettlin Erich,OW,+,Halbkanton,Ja
62770,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Graf Maya,BL,=,Halbkanton,Nein
62758,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Fässler Daniel,AI,+,Halbkanton,Ja
62778,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Herzog Eva,BS,-,Halbkanton,Nein
62744,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Caroni Andrea,AR,+,Halbkanton,Ja
62773,SR_5121_2306.pdf,13.06.2023 08:39:21,23.3448,Ordnungsantrag,Ordnungsantrag,+,Wicki Hans,NW,+,Halbkanton,Ja
39582,SR_5116_2209.pdf,22.09.2022 15:04:05,21.047,Ordnungsantrag,Ordnungsantrag Kuprecht,-,Français Olivier,VD,-,beide dagegen,Ja
39591,SR_5116_2209.pdf,22.09.2022 15:04:05,21.047,Ordnungsantrag,Ordnungsantrag Kuprecht,-,Thorens Goumaz Adèle,VD,-,beide dagegen,Ja
39587,SR_5116_2209.pdf,22.09.2022 15:04:05,21.047,Ordnungsantrag,Ordnungsantrag Kuprecht,-,Stark Jakob,TG,+,beide dafür,Nein
39592,SR_5116_2209.pdf,22.09.2022 15:04:05,21.047,Ordnungsantrag,Ordnungsantrag Kuprecht,-,Häberli-Koller Brigitte,TG,+,beide dafür,Nein


In [267]:
len(df_komplett)

65735

In [234]:
df_komplett.to_excel("Datensatz.xlsx")  

In [235]:
df_komplett.to_csv("Datensatz.csv")  

In [236]:
df_komplett[df_komplett["Politiker"] == "Noser Ruedi"]["Standesstimme"].value_counts()

beide dafür         561
nicht vollzählig    516
uneinig             256
beide absent         59
beide dagegen        42
Name: Standesstimme, dtype: int64

In [262]:
df_komplett[df_komplett["siegreich"] == "Nein"][df_komplett["Standesstimme"] != "uneinig"][df_komplett["Standesstimme"] != "Halbkanton"][df_komplett["Standesstimme"] != "nicht vollzählig"][df_komplett["Standesstimme"] != "beide absent"]["Kanton"].value_counts()

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


GE    522
SH    326
AG    168
VS    126
JU    116
VD     96
FR     94
SZ     80
GR     76
ZG     76
NE     68
LU     58
UR     50
TG     42
SO     42
SG     40
BE     28
TI     28
ZH     28
GL     16
Name: Kanton, dtype: int64

In [251]:
df_komplett[df_komplett["siegreich"] == "Nein"]["Politiker"].value_counts()

Kuprecht Alex                  626
Hefti Thomas                   611
Noser Ruedi                    541
Stöckli Hans                   541
Vara Céline                    507
Mazzone Lisa                   502
Chiesa Marco                   501
Jositsch Daniel                491
Stark Jakob                    490
Häberli-Koller Brigitte        482
Salzmann Werner                456
Herzog Eva                     440
Knecht Hansjörg                431
Rechsteiner Paul               412
Thorens Goumaz Adèle           391
Minder Thomas                  389
Germann Hannes                 388
Sommaruga Carlo                386
Caroni Andrea                  369
Zopfi Mathias                  363
Zanetti Roberto                361
Graf Maya                      355
Schmid Martin                  335
Baume-Schneider Elisabeth      317
Carobbio Guscetti Marina       285
Hegglin Peter                  277
Français Olivier               270
Bischof Pirmin                 266
Maret Marianne      

In [237]:
df_komplett[df_komplett["Politiker"] == "Noser Ruedi"]["Verhalten"].value_counts()

+    829
0    376
-    183
E     30
=     16
Name: Verhalten, dtype: int64

In [238]:
df_komplett[df_komplett["Politiker"] == "Jositsch Daniel"]["Verhalten"].value_counts()

+    938
-    227
0    201
E     56
=     12
Name: Verhalten, dtype: int64

In [239]:
df_komplett[df_komplett["Verhalten"] == "0"]["Kanton"].value_counts()

ZH    577
SZ    366
GR    279
BE    259
SG    253
UR    224
SO    216
TI    207
TG    204
LU    199
FR    189
ZG    187
NE    182
AR    180
AG    169
VS    149
BS    137
SH    126
VD    118
JU    111
OW    110
GL    105
GE    102
NW    101
AI     36
BL     28
Name: Kanton, dtype: int64

In [240]:
df_komplett[df_komplett["Standesstimme"] == "beide absent"]["Politiker"].value_counts()

Noser Ruedi                    59
Jositsch Daniel                59
Engler Stefan                  29
Schmid Martin                  29
Z'graggen Heidi                21
Dittli Josef                   21
Zanetti Roberto                17
Bischof Pirmin                 17
Würth Benedikt                 16
Rechsteiner Paul               16
Müller Damian                  15
Gmür-Schönenberger Andrea      15
Kuprecht Alex                  14
Reichmuth Othmar               14
Gapany Johanna                 11
Rieder Beat                    10
Mazzone Lisa                   10
Sommaruga Carlo                10
Maret Marianne                 10
Hegglin Peter                   9
Burkart Thierry                 9
Knecht Hansjörg                 9
Michel Matthias                 9
Salzmann Werner                 8
Chassot Isabelle                8
Stöckli Hans                    8
Germann Hannes                  7
Minder Thomas                   7
Vara Céline                     6
Bauer Philippe

In [252]:
df_komplett[df_komplett["Standesstimme"] == "beide absent"]["Kanton"].value_counts()

ZH    118
GR     58
UR     42
SO     34
SG     32
LU     30
SZ     28
FR     22
GE     20
VS     20
AG     18
ZG     18
BE     16
SH     14
NE     12
TI     12
TG      8
JU      6
VD      4
GL      2
Name: Kanton, dtype: int64

In [241]:
df_züri = df_komplett[df_komplett["Kanton"] == "ZH"]
df_züri[df_züri["Standesstimme"] == "beide absent"]

,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,Politiker,Kanton,Verhalten,siegreich,Standesstimme
42699,SR_5116_2209.pdf,20.09.2022 08:51:37,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Gesamtabstimmung,+,Noser Ruedi,ZH,0,Nein,beide absent
42726,SR_5116_2209.pdf,20.09.2022 08:51:37,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Gesamtabstimmung,+,Jositsch Daniel,ZH,0,Nein,beide absent
42653,SR_5116_2209.pdf,20.09.2022 08:51:03,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Art. 25a,+,Noser Ruedi,ZH,0,Nein,beide absent
42680,SR_5116_2209.pdf,20.09.2022 08:51:03,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Art. 25a,+,Jositsch Daniel,ZH,0,Nein,beide absent
42561,SR_5116_2209.pdf,13.09.2022 10:42:03,21.4341,"Mo. WBK-NR. Kinderwunsch erfüllen, Eizellenspe...",Abstimmung über die Motion,+,Noser Ruedi,ZH,0,Nein,beide absent
...,...,...,...,...,...,...,...,...,...,...,...
26078,SR_5118_2303.pdf,01.03.2023 08:39:33,23.018-1,Assistenzdienst der Armee zur Unterstützung de...,Gesamtabstimmung,+,Jositsch Daniel,ZH,0,Nein,beide absent
28857,SR_5115_2206.pdf,14.06.2022 11:38:13,20.4078,Mo. Silberschmidt. Netto-null-Ziel im Jahr 205...,Abstimmung über die Motion,+,Noser Ruedi,ZH,0,Nein,beide absent
28884,SR_5115_2206.pdf,14.06.2022 11:38:13,20.4078,Mo. Silberschmidt. Netto-null-Ziel im Jahr 205...,Abstimmung über die Motion,+,Jositsch Daniel,ZH,0,Nein,beide absent
32583,SR_5115_2206.pdf,16.06.2022 18:25:30,22.3067,"Mo. Knecht. Aufhebung des Verbots, Rahmenbewil...",Abstimmung über die Motion,-,Noser Ruedi,ZH,0,Nein,beide absent


In [242]:
df_züri[df_züri["Standesstimme"] == "beide absent"].to_excel("Zuerich_absent.xlsx")  

In [243]:
df_komplett[df_komplett["Standesstimme"] == "uneinig"]["Kanton"].value_counts()

BE    1078
NE     926
TI     880
SO     638
JU     628
AG     620
VD     606
SH     544
SG     542
ZH     512
GL     504
SZ     428
FR     428
TG     414
ZG     404
UR     346
GR     334
VS     328
LU     312
GE     144
Name: Kanton, dtype: int64

In [244]:
df_komplett[df_komplett["siegreich"] == "Ja"]["Kanton"].value_counts()

LU    2437
UR    2405
VS    2370
ZG    2361
FR    2341
GR    2309
SO    2241
VD    2207
AG    2190
JU    2183
NE    2132
SG    2127
SH    2091
SZ    2007
GE    1980
TI    1925
TG    1896
GL    1891
BE    1871
ZH    1836
AI    1226
OW    1218
NW    1188
BL    1079
AR    1065
BS     994
Name: Kanton, dtype: int64